In [23]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import matplotlib.pyplot as plt
import seaborn as sns


In [30]:
libs = ['SRR1197125']
path_libs = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/sun_deg__%s'
path_transcripts = '/home/juan/Desktop/juan/bio/mirna_mite/data/fixed.cdna.all.fa'
path_transcripts_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/targets_mites.fa'
path_blast_res = '/home/juan/Desktop/juan/bio/mirna_mite/data/res/transcripts_mites.csv'

In [31]:
for lib in libs:
    path_lib = path_libs % lib
    df = pd.read_csv(path_lib, sep='\t',comment='#')

In [32]:
df.head(2)

,SiteID,Query,Transcript,TStart,TStop,TSlice,MFEperfect,MFEsite,MFEratio,AllenScore,Paired,Unpaired,Structure,Sequence,DegradomeCategory,DegradomePval,Tplot_file_path
0,TraesCS1A02G033600.1:767,mrcv_all_Cluster_12767,TraesCS1A02G033600.1,753,776,767,-37.0,-24.5,0.662162,10.0,"1-10,776-767;13-15,764-762;16-21,758-753","11-12,766-765[SIL];x-x,761-759[BULt]",((((((...(((..((((((((((&))))))))))..)))---)))))),UGCCCCGUUCUCAUGUCCAUGUUG&UAAUAUGGAUUGGAG---GGGGUA,2,1.000000,/home/juan/Desktop/juan/bio/mirna_mite/data/re...
1,TraesCS1A02G105600.1:1678,mrcv_mites_MITE_T_58879,TraesCS1A02G105600.1,1668,1687,1678,-29.7,-20.4,0.686869,6.5,"1-9,1687-1679;11-12,1677-1676;14-21,1675-1668","10-10,1678-1678[SIL];13-13,x-x[BULq]",((((((((-((.(((((((((&))))))))).)).)))))))),GAUCCACA-UGUGAUUUCUGA&UUAGAGAUUUCAAUGUGGAUU,4,0.994485,/home/juan/Desktop/juan/bio/mirna_mite/data/re...


In [33]:
transcripts = set()
for k,v in df.iterrows():
    transcript = v.Transcript
    transcripts.add(transcript)

In [34]:
fasta_seq = SeqIO.parse(path_transcripts, 'fasta')


In [35]:
buffer_seqs = []
for record in fasta_seq:
    if record.id in transcripts:
        buffer_seqs.append(record)
SeqIO.write(buffer_seqs, path_transcripts_res, "fasta")
print(path_transcripts_res)

/home/juan/Desktop/juan/bio/mirna_mite/data/res/targets_mites.fa


In [49]:
df_mites = pd.read_csv(path_blast_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs','score']
df_mites.columns = cols
df_mites = df_mites[df_mites.pident >= 80]
df_mites = df_mites[df_mites.length >= (df_mites.slen * 0.5)]
print(len(df_mites.index))
df_mites.head(2)

4429


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
3,TraesCS5B02G523000.1,MITE_332,1039,1091,38,90,7,0,86.792,4.210000e-11,53,1339,91,4,70
9,TraesCS5B02G523000.1,DTT_Bdis_BdisStowawayR_consensus-1,1039,1090,35,86,8,0,84.615,1.790000e-09,52,1339,86,4,64


In [51]:
df['new_start'] = df[['TStart','TStop']].min(axis=1)
df['new_end'] = df[['TStart','TStop']].max(axis=1)
df['TStart'] = df['new_start']
df['TStop'] = df['new_end']
df = df.drop('new_start',axis=1).drop('new_end',axis=1)


In [53]:
df_mites['new_start'] = df_mites[['sstart','send']].min(axis=1)
df_mites['new_end'] = df_mites[['sstart','send']].max(axis=1)
df_mites['sstart'] = df_mites['new_start']
df_mites['send'] = df_mites['new_end']
df_mites = df_mites.drop('new_start',axis=1).drop('new_end',axis=1)


In [80]:
df['MITE'] = ''

In [81]:
for k,v in df.iterrows():
    start = v.TStart
    end = v.TStop
    transcript = v.Transcript
    df_filter = df_mites[df_mites.qseqid==transcript]
    df_filter = df_filter[(df_filter.qstart <= start) & (df_filter.qend <= end)]
    if len(df_filter.index) > 0:
        mites = ','.join(df_filter.sseqid.tolist())
        df.at[k, 'MITE'] += mites
        

In [86]:
total = len(df.index)
w_mites = len(df[df.MITE != ''].index)
w_o_mites = len(df[df.MITE == ''].index)
print('Total', total)
print('W mites', w_mites)
print('WO mites', w_o_mites)
print(w_mites * 100 / total)

Total 619
W mites 49
WO mites 570
7.915993537964459


In [87]:
df[df.MITE != ''][['SiteID','Query','Transcript','TStart','TStop','MITE']]

,SiteID,Query,Transcript,TStart,TStop,MITE
2,TraesCS1A02G107800.2:1919,mrcv_mites_MITE_T_121053,TraesCS1A02G107800.2,1909,1928,"MITE_1767,MITE_874"
7,TraesCS1A02G138200.2:1948,sun_all_Cluster_8835,TraesCS1A02G138200.2,1936,1956,"MITE_1314,DTT_Hvul_Athos_EF067844-2,MITE_266"
22,TraesCS1A02G350400.2:1719,mrcv_mites_MITE_T_80536,TraesCS1A02G350400.2,1702,1728,"REP-1_OS,MITE_253"
30,TraesCS1A02G419600.1:2090,mrcv_mites_MITE_T_121053,TraesCS1A02G419600.1,2079,2099,"MITE_955,MITE_955,DTT_Taes_Athos_BQ238527-1,DT..."
32,TraesCS1A02G443600.1:1711,mrcv_mites_MITE_T_121053,TraesCS1A02G443600.1,1701,1720,"DTT_Taes_Thalos_BJ276470-1,DTT_Tdur_Thalos_103..."
53,TraesCS1B02G420400.3:3578,mrcv_mites_MITE_T_58879,TraesCS1B02G420400.3,3567,3586,"DTT_Taes_Thalos_BE405570-1,MITE_928,MITE_1303"
55,TraesCS1B02G434600.1:501,mrcv_mites_MITE_T_80536,TraesCS1B02G434600.1,490,510,"DTT_Taes_Icarus_BQ237274-1,MITE_1745,MITE_1397..."
56,TraesCS1B02G446800.1:721,mrcv_mites_MITE_T_121053,TraesCS1B02G446800.1,710,730,"MITE_955,MITE_955,DTT_Taes_Athos_42j2-5,DTT_Ta..."
63,TraesCS1D02G010200.1:2957,mrcv_mites_MITE_T_80536,TraesCS1D02G010200.1,2940,2966,MITE_147
65,TraesCS1D02G101200.2:1360,mrcv_all_Cluster_13743,TraesCS1D02G101200.2,1349,1368,"MITE_233,MITE_233,MITE_1717,MITE_1180,MITE_1180"
